<a href="https://colab.research.google.com/github/okj3031402/LLM_study/blob/main/building_a_simple_vectorstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 키 설정

In [ ]:
import os

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# 패키지 설치

In [ ]:
!pip install openai # openai 라이브러리를 설치합니다.
!pip install langchain # 랭체인 라이브러리를 설치합니다.
!pip install tqdm
!pip install chromadb # 벡터스토어
!pip install tiktoken # 토큰 계산용
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24

# RAG를 위한 파일 준비

In [ ]:

import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt",
    filename="state_of_the_union.txt"
)

('state_of_the_union.txt', <http.client.HTTPMessage at 0x79738b10d690>)

In [ ]:
from tqdm import tqdm

# SimpleTextLoader 구현해보기

In [ ]:
class SimpleTextLoader:

    def __init__(self, file_path):
        self.file_path = file_path

    def load(self):
        text = ''
        with open(self.file_path, 'r', encoding='utf-8') as file:
            text = file.read()
        return text

# SimpleCharacterTextSplitter 구현해보기

In [ ]:
# 이삿짐 센터
# 여러가지 박스 크기
# 박스를 이사 전에 가져다 줌
# 짐을 박스에 담는다.

class SimpleCharacterTextSplitter:

    def __init__(self, chunk_size, chunk_overlap, separator_pattern='\n\n'):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.separator_pattern = separator_pattern

    def split_documents(self, documents):

        # 파일 전체 내용을 문단 단위로 나눈다.
        splits = documents.split(self.separator_pattern)

        # A split 500자
        # B split 10자
        # C split 600자

        # 위 split들을 청크(박스)에 담는 행위
        # A + B = 510 첫번째 박스
        # C는 600자라서 A + B + C = 1110 > 1000 박스 크기 초과
        # C 두번째 박스
        # 첫번째 박스 (510 : A, B)
        # 두번째 박스 (600 : C)

        chunks = []
        current_chunk = splits[0]

        for split in tqdm(splits[1:], desc="splitting..."):

            if len(current_chunk) + len(split) + len(self.separator_pattern) > self.chunk_size:
                chunks.append(current_chunk.strip())
                current_chunk = split
            else:
                current_chunk += self.separator_pattern
                current_chunk += split

        if current_chunk:
            chunks.append(current_chunk.strip())

        return chunks

# SimpleOpenAIEmbeddings 구현해보기

In [ ]:
from openai import OpenAI

class SimpleOpenAIEmbeddings:

    def embed_query(self, text):
        client = OpenAI()
        response = client.embeddings.create(
            input=text,
            model="text-embedding-ada-002"
        )
        return response.data[0].embedding

# SimpleVectorStore 구현해보기

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

class SimpleVectorStore:
    def __init__(self, docs, embedding):
        self.embedding = embedding
        self.documents = []
        self.vectors = []

        # 벡터 스토어는 벡터만 담고있다 X
        # 벡터는 유사도 판단을 위해서 사용되며,
        # 그 안에 콘텐츠를 가지고 와야되기 때문에, 실제 청크 내용도 가지고 있어야 합니다.

        for doc in tqdm(docs, desc="embedding..."):
            self.documents.append(doc)
            vector = self.embedding.embed_query(doc)
            self.vectors.append(vector)

    def similarity_search(self, query, k=4):
        query_vector = self.embedding.embed_query(query)

        if not self.vectors:
            return []

        similarities = cosine_similarity([query_vector], self.vectors)[0]
        sorted_doc_similarities = sorted(zip(self.documents, similarities), key=lambda x: x[1], reverse=True)

        return sorted_doc_similarities[:k]

    def as_retriever(self, k=4):
        return SimpleRetriever(self, k)

# SimpleRetriever 구현해보기

In [ ]:
class SimpleRetriever:
    def __init__(self, vector_store, k=4):
        self.vector_store = vector_store
        self.k = k

    def get_relevant_documents(self, query):
        docs = self.vector_store.similarity_search(query, self.k)
        return docs

In [ ]:
raw_documents = SimpleTextLoader('state_of_the_union.txt').load()
text_splitter = SimpleCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = SimpleVectorStore(documents, SimpleOpenAIEmbeddings())

embedding...: 100%|██████████| 42/42 [00:12<00:00,  3.26it/s]


In [ ]:
len(documents)

42

In [ ]:
documents[0:4]

['Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.',
 'Groups of citizens blocking tanks with

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [ ]:
docs

[('Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.',
  0.8152526831877118),
 ('A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’

In [ ]:
print(docs[0][0])

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


# 한글 벡터스토어

# 헌법 예시

In [ ]:
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/puzzlet/constitution-kr/master/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%20%ED%97%8C%EB%B2%95.txt",
    filename="korea_constitution.txt"
)

('korea_constitution.txt', <http.client.HTTPMessage at 0x79735ad59990>)

In [ ]:
raw_documents = SimpleTextLoader('korea_constitution.txt').load()
text_splitter = SimpleCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

splitting...: 100%|██████████| 152/152 [00:00<00:00, 391846.47it/s]


In [ ]:
documents

['전문\n\n유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.\n\n제1장 총강\n\n제1조 ① 대한민국은 민주공화국이다.\n② 대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.\n\n제2조 ① 대한민국의 국민이 되는 요건은 법률로 정한다.\n② 국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다.\n\n제3조 대한민국의 영토는 한반도와 그 부속도서로 한다.\n\n제4조 대한민국은 통일을 지향하며, 자유민주적 기본질서에 입각한 평화적 통일 정책을 수립하고 이를 추진한다.\n\n제5조 ① 대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다.\n② 국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다.\n\n제6조 ① 헌법에 의하여 체결·공포된 조약과 일반적으로 승인된 국제법규는 국내법과 같은 효력을 가진다.\n② 외국인은 국제법과 조약이 정하는 바에 의하여 그 지위가 보장된다.\n\n제7조 ① 공무원은 국민전체에 대한 봉사자이며, 국민에 대하여 책임을 진다.\n② 공무원의 신분과 정치적 중립성은 법률이 정하는 바에 의하여 보장된다.',
 '제8조 ① 정당의 설

In [ ]:
db = SimpleVectorStore(documents, SimpleOpenAIEmbeddings())

embedding...: 100%|██████████| 22/22 [00:06<00:00,  3.44it/s]


In [ ]:
query = "대통령 임기는?"
docs = db.similarity_search(query)

In [ ]:
docs

[('제65조 ① 대통령·국무총리·국무위원·행정각부의 장·헌법재판소 재판관·법관·중앙선거관리위원회 위원·감사원장·감사위원 기타 법률이 정한 공무원이 그 직무집행에 있어서 헌법이나 법률을 위배한 때에는 국회는 탄핵의 소추를 의결할 수 있다.\n② 제1항의 탄핵소추는 국회재적의원 3분의 1 이상의 발의가 있어야 하며, 그 의결은 국회재적의원 과반수의 찬성이 있어야 한다. 다만, 대통령에 대한 탄핵소추는 국회재적의원 과반수의 발의와 국회재적의원 3분의 2 이상의 찬성이 있어야 한다.\n③ 탄핵소추의 의결을 받은 자는 탄핵심판이 있을 때까지 그 권한행사가 정지된다.\n④ 탄핵결정은 공직으로부터 파면함에 그친다. 그러나, 이에 의하여 민사상이나 형사상의 책임이 면제되지는 아니한다.\n\n제4장 정부\n제1절 대통령\n\n제66조 ① 대통령은 국가의 원수이며, 외국에 대하여 국가를 대표한다.\n② 대통령은 국가의 독립·영토의 보전·국가의 계속성과 헌법을 수호할 책무를 진다.\n③ 대통령은 조국의 평화적 통일을 위한 성실한 의무를 진다.\n④ 행정권은 대통령을 수반으로 하는 정부에 속한다.\n\n제67조 ① 대통령은 국민의 보통·평등·직접·비밀선거에 의하여 선출한다.\n② 제1항의 선거에 있어서 최고득표자가 2인 이상인 때에는 국회의 재적의원 과반수가 출석한 공개회의에서 다수표를 얻은 자를 당선자로 한다.\n③ 대통령후보자가 1인일 때에는 그 득표수가 선거권자 총수의 3분의 1 이상이 아니면 대통령으로 당선될 수 없다.\n④ 대통령으로 선거될 수 있는 자는 국회의원의 피선거권이 있고 선거일 현재 40세에 달하여야 한다.\n⑤ 대통령의 선거에 관한 사항은 법률로 정한다.\n\n제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.\n② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.',
  0.8401361376422471),
 ('제81조 대통령은 국

# 벡터스토어 튜닝하기

In [ ]:
raw_documents = SimpleTextLoader('korea_constitution.txt').load()
text_splitter = SimpleCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
# 이삿짐 센터, 박스 크기 작다. 열린 상태인 박스에 그냥 물건을 담을 수 있다.
documents = text_splitter.split_documents(raw_documents)

splitting...: 100%|██████████| 152/152 [00:00<00:00, 122390.90it/s]


In [ ]:
documents[0:10]

['전문',
 '유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.',
 '제1장 총강\n\n제1조 ① 대한민국은 민주공화국이다.\n② 대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.',
 '제2조 ① 대한민국의 국민이 되는 요건은 법률로 정한다.\n② 국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다.',
 '제3조 대한민국의 영토는 한반도와 그 부속도서로 한다.\n\n제4조 대한민국은 통일을 지향하며, 자유민주적 기본질서에 입각한 평화적 통일 정책을 수립하고 이를 추진한다.',
 '제5조 ① 대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다.\n② 국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다.',
 '제6조 ① 헌법에 의하여 체결·공포된 조약과 일반적으로 승인된 국제법규는 국내법과 같은 효력을 가진다.\n② 외국인은 국제법과 조약이 정하는 바에 의하여 그 지위가 보장된다.',
 '제7조 ① 공무원은 국민전체에 대한 봉사자이며, 국민에 대하여 책임을 진다.\n② 공무원의 신분과 정치적 중립성은 법률이 정하는 바에 의하여 보장된다.',
 '제8조 

In [ ]:
db = SimpleVectorStore(documents, SimpleOpenAIEmbeddings())

embedding...: 100%|██████████| 137/137 [00:32<00:00,  4.20it/s]


In [ ]:
query = "대통령 임기는 몇 년인가?"
docs = db.similarity_search(query)

In [ ]:
docs

[('제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.', 0.8773323150496815),
 ('제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.\n② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.',
  0.8643703240016223),
 ('제105조 ① 대법원장의 임기는 6년으로 하며, 중임할 수 없다.\n② 대법관의 임기는 6년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.\n③ 대법원장과 대법관이 아닌 법관의 임기는 10년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.\n④ 법관의 정년은 법률로 정한다.',
  0.8540177946936056),
 ('제42조 국회의원의 임기는 4년으로 한다.\n\n제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.',
  0.8484746328973443)]

In [ ]:
query = "대통령 임기는?"
docs = db.similarity_search(query)

In [ ]:
docs

[('제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.\n② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.',
  0.8563717346900667),
 ('제4장 정부\n제1절 대통령', 0.8525658918880346),
 ('제69조 대통령은 취임에 즈음하여 다음의 선서를 한다.\n"나는 헌법을 준수하고 국가를 보위하며 조국의 평화적 통일과 국민의 자유와 복리의 증진 및 민족문화의 창달에 노력하여 대통령으로서의 직책을 성실히 수행할 것을 국민 앞에 엄숙히 선서합니다."',
  0.8517005070139246),
 ('제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.', 0.8503461998892345)]

# 검색기 만들기

In [ ]:
retriever = db.as_retriever()

In [ ]:
unique_docs = retriever.get_relevant_documents(query="대통령의 임기는 몇 년인가?")

In [ ]:
unique_docs

[('제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.', 0.8802913492732869),
 ('제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.\n② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.',
  0.8671480195134746),
 ('제105조 ① 대법원장의 임기는 6년으로 하며, 중임할 수 없다.\n② 대법관의 임기는 6년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.\n③ 대법원장과 대법관이 아닌 법관의 임기는 10년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.\n④ 법관의 정년은 법률로 정한다.',
  0.8549414951338008),
 ('제98조 ① 감사원은 원장을 포함한 5인 이상 11인 이하의 감사위원으로 구성한다.\n② 원장은 국회의 동의를 얻어 대통령이 임명하고, 그 임기는 4년으로 하며, 1차에 한하여 중임할 수 있다.\n③ 감사위원은 원장의 제청으로 대통령이 임명하고, 그 임기는 4년으로 하며, 1차에 한하여 중임할 수 있다.',
  0.8506226670903798)]

# 챗봇 만들기

In [ ]:
import openai

system_prompt_template = ("You are a helpful assistant. "
                          "Based on the following content, "
                          "kindly and comprehensively respond to user questions. write in Korean."
                          "[Content]"
                          "{content}"
                          "")

class SimpleRetrievalQA():

    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, query):
        docs = self.retriever.get_relevant_documents(query) # 검색 > R
        print(docs)

        for i, doc in enumerate(docs):
            print("[#" + str(i) + "]", doc[1])
            print(doc[0])

        completion = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt_template.format(content=docs)}, # 증강 > A
                {"role": "user", "content": query}
            ]
        ) # 생성 > G

        return completion.choices[0].message.content

In [ ]:
chain = SimpleRetrievalQA(retriever)

answer = chain.invoke("대통령의 임기는?")

print(">> ", answer)

[('제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.\n② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.', 0.8630652979839899), ('제69조 대통령은 취임에 즈음하여 다음의 선서를 한다.\n"나는 헌법을 준수하고 국가를 보위하며 조국의 평화적 통일과 국민의 자유와 복리의 증진 및 민족문화의 창달에 노력하여 대통령으로서의 직책을 성실히 수행할 것을 국민 앞에 엄숙히 선서합니다."', 0.8563476626121538), ('제4장 정부\n제1절 대통령', 0.8562722026140897), ('제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.', 0.853958628309929)]
[#0] 0.8630652979839899
제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.
② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.
[#1] 0.8563476626121538
제69조 대통령은 취임에 즈음하여 다음의 선서를 한다.
"나는 헌법을 준수하고 국가를 보위하며 조국의 평화적 통일과 국민의 자유와 복리의 증진 및 민족문화의 창달에 노력하여 대통령으로서의 직책을 성실히 수행할 것을 국민 앞에 엄숙히 선서합니다."
[#2] 0.8562722026140897
제4장 정부
제1절 대통령
[#3] 0.853958628309929
제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
>>  대통령의 임기는 5년으로 하며, 중임할 수 없습니다. (제70조)


In [ ]:
chain = SimpleRetrievalQA(retriever)

answer = chain.invoke("대통령은 중임할 수 있나요?")

print(">> ", answer)

[('제72조 대통령은 필요하다고 인정할 때에는 외교·국방·통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수 있다.', 0.8679194676690196), ('제69조 대통령은 취임에 즈음하여 다음의 선서를 한다.\n"나는 헌법을 준수하고 국가를 보위하며 조국의 평화적 통일과 국민의 자유와 복리의 증진 및 민족문화의 창달에 노력하여 대통령으로서의 직책을 성실히 수행할 것을 국민 앞에 엄숙히 선서합니다."', 0.8635573299429514), ('제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.', 0.8631142710856652), ('제76조 ① 대통령은 내우·외환·천재·지변 또는 중대한 재정·경제상의 위기에 있어서 국가의 안전보장 또는 공공의 안녕질서를 유지하기 위하여 긴급한 조치가 필요하고 국회의 집회를 기다릴 여유가 없을 때에 한하여 최소한으로 필요한 재정·경제상의 처분을 하거나 이에 관하여 법률의 효력을 가지는 명령을 발할 수 있다.\n② 대통령은 국가의 안위에 관계되는 중대한 교전상태에 있어서 국가를 보위하기 위하여 긴급한 조치가 필요하고 국회의 집회가 불가능한 때에 한하여 법률의 효력을 가지는 명령을 발할 수 있다.\n③ 대통령은 제1항과 제2항의 처분 또는 명령을 한 때에는 지체없이 국회에 보고하여 그 승인을 얻어야 한다.\n④ 제3항의 승인을 얻지 못한 때에는 그 처분 또는 명령은 그때부터 효력을 상실한다. 이 경우 그 명령에 의하여 개정 또는 폐지되었던 법률은 그 명령이 승인을 얻지 못한 때부터 당연히 효력을 회복한다.\n⑤ 대통령은 제3항과 제4항의 사유를 지체없이 공포하여야 한다.', 0.8624341764536383)]
[#0] 0.8679194676690196
제72조 대통령은 필요하다고 인정할 때에는 외교·국방·통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수 있다.
[#1] 0.8635573299429514
제69조 대통령은 취임에 즈음하여 다음의 선서를 한다.
"나는 헌법을 준수하고 국가를 보위하며 

In [ ]:
def chat_with_user(user_message):
    ai_message = chain.invoke(user_message)
    return ai_message

while True:
    user_message = input("USER > ")
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message)
    print(f" A I > {ai_message}")

USER > quit


In [ ]:
retriever = db.as_retriever(k=3)
chain = SimpleRetrievalQA(retriever)
answer = chain.invoke("대통령의 임기는?")

print(">> ", answer)

# 벡터스토어의 성능이 높다면, 청크 개수를 줄여도 좋습니다.

[('제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.\n② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.', 0.8629479744559038), ('제69조 대통령은 취임에 즈음하여 다음의 선서를 한다.\n"나는 헌법을 준수하고 국가를 보위하며 조국의 평화적 통일과 국민의 자유와 복리의 증진 및 민족문화의 창달에 노력하여 대통령으로서의 직책을 성실히 수행할 것을 국민 앞에 엄숙히 선서합니다."', 0.8562098852834971), ('제4장 정부\n제1절 대통령', 0.8561517726078522)]
[#0] 0.8629479744559038
제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.
② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.
[#1] 0.8562098852834971
제69조 대통령은 취임에 즈음하여 다음의 선서를 한다.
"나는 헌법을 준수하고 국가를 보위하며 조국의 평화적 통일과 국민의 자유와 복리의 증진 및 민족문화의 창달에 노력하여 대통령으로서의 직책을 성실히 수행할 것을 국민 앞에 엄숙히 선서합니다."
[#2] 0.8561517726078522
제4장 정부
제1절 대통령
>>  대통령의 임기는 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거하며, 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거합니다.


# 로컬 임베딩 모델

In [ ]:
import langchain
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
raw_documents = SimpleTextLoader('korea_constitution.txt').load()
text_splitter = SimpleCharacterTextSplitter(chunk_size=10, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

embed_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")
print(embed_model)
db = SimpleVectorStore(documents, embed_model)

splitting...: 100%|██████████| 152/152 [00:00<00:00, 132709.04it/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
) model_name='jhgan/ko-sbert-sts' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


embedding...: 100%|██████████| 153/153 [00:45<00:00,  3.35it/s]


In [ ]:
query = "대통령의 임기는?"
docs = db.similarity_search(query)

In [ ]:
docs

[('제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.', 0.538174621737578),
 ('제105조 ① 대법원장의 임기는 6년으로 하며, 중임할 수 없다.\n② 대법관의 임기는 6년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.\n③ 대법원장과 대법관이 아닌 법관의 임기는 10년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.\n④ 법관의 정년은 법률로 정한다.',
  0.4657673348720374),
 ('제42조 국회의원의 임기는 4년으로 한다.', 0.43761092203517576),
 ('제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.\n② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.',
  0.4304336255208122)]

In [ ]:
retriever = db.as_retriever(k=5)
chain = SimpleRetrievalQA(retriever)
answer = chain.invoke("대통령의 임기는?")

print(">> ", answer)

[('제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.', 0.538174621737578), ('제105조 ① 대법원장의 임기는 6년으로 하며, 중임할 수 없다.\n② 대법관의 임기는 6년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.\n③ 대법원장과 대법관이 아닌 법관의 임기는 10년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.\n④ 법관의 정년은 법률로 정한다.', 0.4657673348720374), ('제42조 국회의원의 임기는 4년으로 한다.', 0.43761092203517576), ('제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.\n② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한 때에는 60일 이내에 후임자를 선거한다.', 0.4304336255208122), ('제128조 ① 헌법개정은 국회재적의원 과반수 또는 대통령의 발의로 제안된다.\n② 대통령의 임기연장 또는 중임변경을 위한 헌법개정은 그 헌법개정 제안 당시의 대통령에 대하여는 효력이 없다.', 0.4221872544484392)]
[#0] 0.538174621737578
제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
[#1] 0.4657673348720374
제105조 ① 대법원장의 임기는 6년으로 하며, 중임할 수 없다.
② 대법관의 임기는 6년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.
③ 대법원장과 대법관이 아닌 법관의 임기는 10년으로 하며, 법률이 정하는 바에 의하여 연임할 수 있다.
④ 법관의 정년은 법률로 정한다.
[#2] 0.43761092203517576
제42조 국회의원의 임기는 4년으로 한다.
[#3] 0.4304336255208122
제68조 ① 대통령의 임기가 만료되는 때에는 임기만료 70일 내지 40일전에 후임자를 선거한다.
② 대통령이 궐위된 때 또는 대통령 당선자가 사망하거나 판결 기타의 사유로 그 자격을 상실한

In [ ]:
def chat_with_user(user_message):
    ai_message = chain.invoke(user_message)
    return ai_message

while True:
    user_message = input("USER > ")
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message)
    print(f" A I > {ai_message}")